## Final Project Submission

* Student name: Ismael Araujo
* Student pace: full time
* Scheduled project review date/time: October 2, 2020
* Instructor name: Fangfang
* Blog post URL:


# Overview
- The purpose of this project is identify the type of movies are currently doing the best in the box office and draw recommendations for a movie studio based on relevant data
- The approach was use multiple datasets to analyse group of genres as well as individual genres, budget, box office, and ROI.

# Business Problem

Microsoft sees all the big companies creating original video content, and they want to get in on the fun. They have decided to create a new movie studio, but the problem is they don’t know anything about creating movies. They have hired you to help them better understand the movie industry. 

<strong>Your team is charged with exploring what type of films are currently doing the best at the box office</strong>. You must then translate those findings into actionable <strong>insights</strong> that the head of Microsoft's new movie studio can use to help <strong>decide what type of films to create</strong>.

# Questions:

1. Is there any connection between the top 20 most profitable movies of all times?
2. Does higher budget reflect in higher profits?
3. What are the top 10 genres with the most movies launched since 1990?
4. What groups of genres bring the highest box office revenue and ROI?
5. What genres bring the highest box office revenue and ROI?

# Sections Summary:

1. Importing Libraries and Datasets
2. Initial Data Wrangling
3. Cleaning the Data
4. Exploring the Data
5. Genres With The Highest Release Of Movies
6. Correlation Between a Higher Budget and Higher Profits
7. Other Correlations
8. Most Profitable Genres Group
9. Most Profitable Individual Genres
10. Final Takeaways and Recomendations

# 1. Importing Libraries and Datasets

### 1.1 Import libraries

In [1]:
# Install xlrd to use the read_excel method
!pip install xlrd

# Import libraries
import pandas as pd 
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter
# This will force Pandas to show all the DataFrame's rows
pd.set_option('display.max_columns', None)

### 1.2 Import files needed to investigate the data

In [ ]:
# Import files that I'll need to investigate the data
df_basics = pd.read_csv('zippedData/imdb.title.basics.csv.gz', compression='gzip')
df_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz', compression='gzip')
df_akas = pd.read_csv('zippedData/imdb.title.akas.csv.gz', compression='gzip')
df_imdb = pd.read_excel('zippedData/movies_metadata.xlsx')
df_movies = pd.read_excel('zippedData/IMDb-movies.xlsx')

### Findings:

- One of the data sets is taking a few seconds to load. I'll filter out the irrelevant data to improve the speed.

### Next Steps:

- Start investigating the DataFrames.

# 2. Initial Data Wrangling

- Open the DataFrames and understand their dimensions and what type of data is available

Note: The cells in this section DOES NOT affect other sections of this notebook.

### 2.1 imdb.title.basics.csv imported as df_basics

In [ ]:
# Start investigating the data sets using .head(1) to see only the first row of the DataFrame
df_basics.head(1)

In [ ]:
# Using .shape to see understand the DataFrame's dimmensions
df_basics.shape

### 2.2 imdb.title.ratings.csv  imported as df_ratings

In [ ]:
df_ratings.head(1)

In [ ]:
df_ratings.shape

### 2.3 imdb.title.ratings.csv.gz imported as df_akas

In [ ]:
df_akas.head(1)

In [ ]:
df_akas.shape

### 2.4 movies_metadata.xlsx imported as df_imdb

In [ ]:
df_imdb.head(1)

In [ ]:
df_imdb.shape

In [ ]:
df_imdb[df_imdb['original_title'] =='Titanic']

### 2.5 IMDb-movies.xlsx imported as df_movies

In [ ]:
df_movies.head(1)

In [ ]:
df_movies.shape

### Findings:

- The data I need is spread in multiple DataFrames
- The IMDb data sets have an ID number, which will help me to merge my the DataFrames
- There are multiple columns will not be needed and multiple rows with empty values
- The df_movies and df_imdb DataFrames is taking a few seconds to load

### Next Steps:

- Since Microsoft is an American company, only movies made by American studios will be used.

# 3. Cleaning the Data

- Rename columns
- Set new indexes
- Merge DataFrames
- Remove data that I will not need

### 3.1 Rename Columns

- Create a pattern between the DataFrames that contains a column with an IMDb ID number and have named them imdb_id

In [ ]:
# Use .rename to rename columns
df_basics.rename(columns={'tconst':'imdb_id'}, inplace=True)
df_akas.rename(columns={'title_id':'imdb_id'}, inplace=True)
df_ratings.rename(columns={'tconst':'imdb_id'}, inplace=True)
df_movies.rename(columns={'imdb_title_id':'imdb_id'}, inplace=True)

### 3.2 Set and match indexes
- Set all the imdb_id columns as an index. It will help when these DataFrames are merged.

In [ ]:
# Use .set_index to set imdb_id as the new index
df_basics.set_index('imdb_id', inplace=True)
df_ratings.set_index('imdb_id', inplace=True)
df_akas.set_index('imdb_id', inplace=True)
df_imdb.set_index('imdb_id', inplace=True)
df_movies.set_index('imdb_id', inplace=True)

### 3.3 Join DataFrames

In [ ]:
# Joining the df_basics and df_akas dataframes. 'df_ba' stands for DataFrame Basics & Akas
df_ba = df_basics.join(df_akas)
df_ba.head(1)

In [ ]:
# Joining the df_ba and df_ratings dataframes. 'bar' stands for DataFrame Basics, Akas & Ratings
df_bar = df_ba.join(df_ratings)
df_bar.head(1)

### 3.4 Drop all movies that weren't produced by studios in the US

Note: Since Microsoft is an American company and the US is the largest market for movies, I will work only with movies produced in the US.

In [ ]:
# Using print(x.shape) to see the dimmensions of more than one DataFrame at once
print(df_bar.shape)
# Checking the size of the DataFrame if I filter all the rows that don't contain 'US' under region
print(df_bar[df_bar.region == 'US'].shape)

In [ ]:
# Filtering out all rows that don't contain 'US' under 'region'
df_bar = df_bar[df_bar.region == 'US']
# Checking the shape. It matches to what was predicted in the cell above
df_bar.shape

In [ ]:
print(df_bar.shape)
print(df_movies.shape)
# Checking the size of the DataFrame if I filter all the rows that don't contain 'USA' under country
print(df_movies[df_movies.country == 'USA'].shape)

In [ ]:
# Filtering out all rows that don't contain 'US' under 'region'
df_movies = df_movies[df_movies.country == 'USA']
df_movies.shape

In [ ]:
# Joining df_movies and df_bar and using rsuffic to avoid conflicts between columns with the same name
df_mbar = df_movies.join(df_bar,rsuffix='_bar')

In [ ]:
# Checking if it worked
df_mbar.head(1)

In [ ]:
# Checking the size of the DataFrame
df_mbar.shape

Note: the df_imdb has multiple columns with the same name as df_bar. Thus, I create another data frame with the values needed before merging it with my df_bar data frame.

In [ ]:
# Create new dataframe with the columns that are relevant. df_br stands for DataFrame Budget & Revenue
df_br = df_imdb.filter(['imdb_id','budget','revenue'], axis=1)
df_br.head(1)

In [ ]:
# This DataFrame will be the main one. Using rsuffix to avoid conflict between columns with the same name
df = df_mbar.join(df_br,rsuffix='_br')
df.head(1)

### 3.5 Add data to NaN values 

The objective is to use fillna to add values to rows with NaN values.

In [ ]:
# Checking null values in the DataFrame
df.isna().sum()

In [ ]:
# Using .fillna() to replace null values with values from another columns
df.budget.fillna(df.budget_br,inplace=True)
df.worlwide_gross_income.fillna(df.revenue,inplace=True)
df.budget_br.fillna(df.budget,inplace=True)
df.revenue.fillna(df.worlwide_gross_income,inplace=True)
df.genres.fillna(df.genre,inplace=True)
df.usa_gross_income .fillna(df.worlwide_gross_income,inplace=True)

In [ ]:
# Checking if fillna() worked
df.isna().sum()

Great! Now we have only movies produced by American studios available for analysis. 

### 3.6 Drop columns

Note: I decided to drop the columns 'primary_title' and 'original_title' because their meanings sound ambiguous. The column 'title' is more straight forward.

In [ ]:
# Checking what columns I have
df.columns

In [ ]:
# Dropping unnecessary data
df.drop(columns=['title','director','writer','actors','description',\
                 'primary_title','original_title_bar','start_year','language','ordering',\
                 'title_bar','region','language_bar','types','attributes','is_original_title',\
                 'budget_br','revenue','runtime_minutes','numvotes','votes', 'reviews_from_users',\
                 'reviews_from_critics','averagerating','date_published'
                 
                ], inplace=True)

In [ ]:
# Checking if it worked
df.head(1)

In [ ]:
# Checking the new DataFrame shape
df.shape

In [ ]:
# Drop duplicated indexes
df = df[~df.index.duplicated(keep='last')]

In [ ]:
# Checking new shape of the DataFrame
df.shape

# 4. Exploring the Data
- New movies throughout the year

### 4.1 First visualization 
- Use data visualization tools to understand the datasets better and if there are outliers that could break the analysis.



In [ ]:
# Using a DataFrame histogram plot to better visualize the dimentions of the values in the DataFrame
df.hist(color='skyblue',edgecolor='black',figsize= (12,12));

In [ ]:
# Checking the types of data
df.dtypes

### 4.2 Use only movies launched between 1990 and 2020

In the 1990s, there was a <a href='https://theface.com/culture/do-the-right-thing-to-the-matrix-how-the-nineties-changed-cinema-forever'>revolution in the movie industry that changed the sector forever</a>. With new technologies, we can create movies in CGI and many other new technologies. It was a new era in the movies. Thus, it's safe to say that we will have a scenario closer to the actual scenario if we use movies produced between 1990 and 2020.

In [ ]:
# Checking the timeframe of my DataFrame
print(df.year.min())
print(df.year.max())

In [ ]:
# Creating a histogram with the start_year columns
x = df.year
plt.hist(x, bins=10,edgecolor='black')
plt.xlabel('Year')
plt.ylabel('Number of New Movies')
plt.show()

In [ ]:
# Filtering my DataFrame to only have movies launched between 1990 and 2020
df = df[df.year >= 1990]
print(df.year.min())
print(df.year.max())

In [ ]:
# Plotting a histogram to visualize when the movies were launched
x = df.year
plt.hist(x, bins=10,edgecolor='black')
plt.xlabel('Year')
plt.ylabel('Number of New Movies')
plt.show()

# 5. Genres With The Highest Number of Movies Released

### Objectives:
- Answer my first question: What genres have the highest release of movies between 1990 and 2020
- Drop NaN values from the genres column
- Separate the genre column that contains more that one genre

### 5.1 Drop NaN values under the genres column

In [ ]:
# Droping rows with missing values in the genres column
df.dropna(subset=['genres'], inplace=True)  

Now that the null values under the genres(last column) column are deleted, I noticed that there are movies with more than one genre. I'll separate them into three columns. I'll use these columns to create a dictionary that will show me how many movies were launched for each value.
I will keep the column genre (4th column) as it is for now. It will be used to further analysis.

### 5.2 Separate values under the genres columns into different columns

In [ ]:
# Separanting the genres columns into three genres using the comma as a dividing point
df[['genre_1','genre_2','genre_3']] =  df.genre.str.split(',',expand=True,)
df.head(1)

In [ ]:
# Counting null values in the new genres columns
print(df.genre_1.isna().sum())
print(df.genre_2.isna().sum())
print(df.genre_3.isna().sum())

Awesome. The genres are separated.

### 5.3 Create a dictionary that counts genres

In [ ]:
# Using groupby to turn the new genres columns into a dictionary
genre_1 = dict(df.groupby(['genre_1']).size())
genre_2 = dict(df.groupby(['genre_2']).size())
genre_3 = dict(df.groupby(['genre_3']).size())

In [ ]:
# Using Counter to sum the dictionary values together
genres_dict = dict(Counter(genre_1)+Counter(genre_2)+Counter(genre_3))
genres_dict

### 5.4 Create a bar plot for the top genres

In [ ]:
# Setting up x and y values using the dictionaries keys and values
x2 = genres_dict.keys()
y2 = genres_dict.values()

In [ ]:
# Defining the figure size
plt.figure(figsize=(30,10))

# Using the bar() function to create a plot using the above values of x and y; and creating a label
plt.bar(x2,y2, label='Genres')

# Creating x and y labels
plt.xlabel('Genres')
plt.ylabel('Number of Movies Launched')

# Creating a title to the label
plt.title('Movies Launched With Each Genre')
plt.legend()

# Final plot
plt.show()

In [ ]:
# Using Counter to check the largest values in my genres dictionary
k = Counter(genres_dict) 
  
# Finding 15 highest values using most_commom
genres_count = k.most_common(15)  

# Printing the top 15 most produced movies genres
print("The top 15 genres with the most movies\nlaunched between 1990 and 2020:\n") 

# Using for loop to print the list in a more readable format
for x,y in genres_count: 
    print(x,'-',y) 

Now, I will plot the top 10 genres with more movies launched.

In [ ]:
# Creating a variable using most_common and then converting it into a dictionary
top_10_genres = k.most_common(10)  
top_10_genres = dict(top_10_genres)

# Setting x and y up
x3 = top_10_genres.keys()
y3 = top_10_genres.values()

# Setting the figure size
plt.figure(figsize=(12,5))

# Using the bar() function to create a plot using the above values of x and y; and create labels
plt.bar(x3,y3, label='Number of Genres Lauched')
plt.xlabel('Genres')
plt.ylabel('Number of Movies Launched')
plt.savefig('destination_path.eps', format='eps')

plt.title('Number of Movies Launched Based on Genre')
plt.legend()

# Output the final plot
plt.show()

### Findings:

- The top 15 genres have the highest release of movies between 1990 and 2020 are Drama, Comedy, Horror, Thriller, Action, Biography, Adventure, Romance, Crime, History, Family, Mystery, Sci-Fi, and Music.



### Next Steps:

- Find out which are the most profitable genres.

# 6. Correlation Between Higher Budget and Higher Profits

Note: Accordingly to <a href='https://theweek.com/articles/647394/when-buy-movie-ticket-where-does-that-money'>this article</a>, studios get approximately 60% of the box office in the US. Thus, I will remove 40% of the final profit.

### 6.1 Check top movies with the highest budget

In [ ]:
# First, I want to take a look of the top budget movies
df.nlargest(5, 'budget')

In [ ]:
# Creates a dictionary with the top 5 movies with the highest budget
df_top_budget = df.filter(['original_title','budget'], axis=1)
df_top_budget.set_index('original_title',inplace=True)
budget_dict = dict(df_top_budget.nlargest(10, 'budget'))
budget_dict

In [ ]:
# The code bellow plots the top 5 highest budgets
top_budget = df.nlargest(5,'budget')
top_budget['budget_million'] = top_budget['budget']/1000000
plt.figure(figsize=(18, 5))
plt.ticklabel_format(style='plain', axis='y')
budget = sns.barplot(x='original_title', y='budget_million', data=top_budget,palette="mako")
budget.set_title('Movies With the Highest Budget', fontdict={'fontsize':16});
budget.set(xlabel='Title', ylabel='Budget in Millions of Dollars')

### 6.2 Drop null values from the budget and worldwide_gross_income columns

In [ ]:
# Dropping NaN values from the budget and worlwide_gross_income columns
df.dropna(subset=['budget','worlwide_gross_income'], inplace=True)  

In [ ]:
# Checking the shape of the dataset after I have dropped the NaN values
df.shape

In [ ]:
# Dropping budget values that are equal to 0
df = df[df['budget'] != 0]

In [ ]:
# Checking the shape of the dataset after I have dropped the 0 values
df.shape

### 6.3 Create a profit column

In [ ]:
# Create column profit. Profit will be worlwide_gross_income minus budget
df['profit'] = (df['worlwide_gross_income'] - df['budget'])

In [ ]:
df.head()

<strong>IMPORTANT:</strong> Studios only take 60% of the sales ticket revenue. Thus, I am removing 40% of each value under the world_gross_income column.

In [ ]:
# This lambda function takes each value under the column 'profit' and multiplies it by 0.6.
df['profit'] = df['profit'].apply(lambda x : x*0.6 if x > 0 else x)

In [ ]:
# Making sure that the worlwide_gross_income and profit columns are integers
df.worlwide_gross_income.astype('int64', copy=True)
df.profit.astype(int).head()

In [ ]:
# Movies with the highest profits
df.nlargest(5, 'profit')

In [ ]:
# Creates a barplot with the top 5 movies with the highest profits
top_profits = df.nlargest(5,'profit')
# Create a profit in millions of dollars
top_profits['profit_million'] = top_profits['profit']/1000000
top_profits['gross_income_million'] = top_profits['worlwide_gross_income']/1000000
plt.figure(figsize=(13, 5))
plt.ticklabel_format(style='plain', axis='y')
plt.savefig('destination_path.eps', format='eps')
top_profits = sns.barplot(x='original_title', y='profit_million', data=top_profits,palette="mako")
top_profits.set_title('Most Profitable Movies', fontdict={'fontsize':16});
top_profits.set(xlabel='Title', ylabel='Profit in Millions of Dollars')

In [ ]:
# Creates a dictionary and a for loop with the top 5 movies with the highest profit
df_top_profit = df.filter(['original_title','profit'], axis=1)
df_top_profit.set_index('original_title',inplace=True)
profit_dict = dict(df_top_profit.nlargest(10, 'profit')['profit'])
profit_dict

print('Most Profitable Movies (in Millions of dollars)\n')
for x,y in profit_dict.items():
    print(x,'-',y/1000000) 

In [ ]:
# Creates a barplot with the top 5 movies with the smallest profit
top_profits = df.nsmallest(5,'profit')
top_profits['profit_million'] = top_profits['profit']/1000000
top_profits['gross_income_million'] = top_profits['worlwide_gross_income']/1000000
plt.figure(figsize=(13, 5))
plt.ticklabel_format(style='plain', axis='y')
plt.savefig('destination_path.eps', format='eps')
top_profits = sns.barplot(x='original_title', y='profit_million', data=top_profits,palette="rocket")
top_profits.set_title('Movies With Highest Loses', fontdict={'fontsize':16});
top_profits.set(xlabel='Title', ylabel='Loses in Millions of Dollars')

In [ ]:
# Creates a dictionary with the top 5 movies with the smallest profit
df_top_profit = df.filter(['original_title','profit'], axis=1)
df_top_profit.set_index('original_title',inplace=True)
profit_dict = dict(df_top_profit.nsmallest(10, 'profit')['profit'])
profit_dict

### 6.4 Correlatalion between budget and higher profits

In [ ]:
# Using a scatter plot to see the relation between budget and profits
plt.figure(figsize=(18, 6))
plt.savefig('destination_path.eps', format='eps')
plt.ticklabel_format(style='plain', axis='y')
plt.ticklabel_format(style='plain', axis='x')
ax = sns.regplot(x="budget", y="worlwide_gross_income",data=df)
ax.set_title('Correlation Between Budget and Box Office', fontdict={'fontsize':12});
ax.set(xlabel='Budget', ylabel='Worldwide Gross Income')

In [ ]:
# Analysing the correlation between budget and profits
budget_revenue_corr = df.budget.corr(df.worlwide_gross_income)
budget_revenue_corr

In [ ]:
top_profits = df.nsmallest(10,'profit')
top_profits['profit_million'] = top_profits['profit']/1000000
top_profits['budget_million'] = top_profits['budget']/1000000
plt.savefig('destination_path.eps', format='eps')
ax = sns.catplot(x="budget_million", y="profit_million", hue="original_title", data=top_profits)
ax.set(xlabel='Budget in Millions of Dollars', ylabel='Losses in Millions of Dollars')

<strong>Takeaways:</strong> We can say that a higher budget has a strong positive correlation with higher profits.

# 7. Other Correlations

In [ ]:
# Increase the size of the heatmap
plt.figure(figsize=(12, 6))
plt.savefig('destination_path.eps', format='eps')
# Store the heatmap variable to use easily when applying other features
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG',annot_kws={"size": 12})
# This fixes a randoring problem on the top and bottom of the heatmap
plt.ylim([7,0])
plt.style.use('seaborn')
# Adding a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=14);

<strong>Takeaways:</strong> 

- Higher budget and higher profits have a moderate correlation
- Budget and box office have a strong correlation
- Duration and average vote have a moderate correlation
- Budget and duration have a moderate correlation

# 8. Most Profitable Genres Group

- Find the most profitable individual genre and groups of genres - net and per dollar invested

Note: The profit column values represent the subtraction of the values in the worlwide_gross_income column by the values under the budget column minus 40%, which is the revenue margin destined to movie theaters.

### 8.1 Make a copy of the main DataFrame

In [ ]:
# Dropping extra genres columns
df.drop(columns=['genres', 'genre_1', 'genre_2','genre_3'], inplace=True)

In [ ]:
# Making a copy of the the main DataFrame in case something goes wrong
df_genre_mean = df.copy()

In [ ]:
# Checking if the copy was successful
df_genre_mean.head(2)

### 8.2 Create a new data frame

A new DataFrame will be used to easily access the mean values for the groups of the genre. The new DataFrame will have values divided by 1,000,000. The objective is to make the readiness easy to be done.

In [ ]:
# This code allows pandas to show all the rows
pd.set_option('display.max_rows', None)
# Group data by genre and get mean for each genre and each variable, divide by 1 million for better visibility
df_genre_mean = df_genre_mean.groupby(['genre']).mean()
df_genre_mean ['profit_million'] = df_genre_mean['profit']/1000000
df_genre_mean['return_per_dollar'] = df_genre_mean['profit']/df_genre_mean['budget']
# Delete the 'profit' column
del df_genre_mean['profit']
df_genre_mean['worlwide_gross_income'] = df_genre_mean['worlwide_gross_income']/1000000
# Delete the 'worlwide_gross_income' column
del df_genre_mean['worlwide_gross_income']
df_genre_mean['budget_million'] = df_genre_mean['budget']/1000000
# Delete the 'budget' column
del df_genre_mean['budget']
# Delete the 'year' column
del df_genre_mean['year']

In [ ]:
# Checking if the new DataFrame is working properly
df_genre_mean.head()

### 8.3 Most profitable group of genres (net)

In [ ]:
# Sorting values by the profit_million column. It's in descending order.
df_genre_mean.sort_values(by=['profit_million'],ascending=False).head(10)

In [ ]:
# Dictionary with the group of genres with the highest profits
dict(df_genre_mean.nlargest(5,'profit_million'))['profit_million']

In [ ]:
genre_group_net = df_genre_mean.nlargest(5,'profit_million')
plt.figure(figsize=(12, 5))
plt.savefig('destination_path.eps', format='eps')
genre_group_net = sns.barplot(x='genre', y='profit_million', data=genre_group_net.reset_index(),palette="mako")
genre_group_net.set_title('Most Profitable Group of Genres', fontdict={'fontsize':14});
genre_group_net.set(xlabel='Groups of Genres', ylabel='Profit In Millions')

### 8.4 Return per dollar invested

In [ ]:
# Sorting values by the return_per_dollar column. It's in descending order.
df_genre_mean.sort_values(by=['return_per_dollar'],ascending=False).head(10)

In [ ]:
dict(df_genre_mean.nlargest(5,'return_per_dollar'))['return_per_dollar']

In [ ]:
genre_group_roi = df_genre_mean.nlargest(5,'return_per_dollar')
plt.figure(figsize=(12, 5))
plt.savefig('destination_path.eps', format='eps')
genre_group_roi = sns.barplot(x='genre', y='return_per_dollar', data=genre_group_roi.reset_index(),palette="mako")
genre_group_roi.set_title('Most Profitable Group of Genre Per Dollar Invested ', fontdict={'fontsize':14});
genre_group_roi.set(xlabel='Groups od Genre', ylabel='Profit Per Dollar Invested')

In [ ]:
# Sorting values by the return_per_dollar column. It's in descending order.
df_genre_mean.sort_values(by=['return_per_dollar'],ascending=True).head(5)

# 9. Most Profitable Individual Genres

In [ ]:
# Make a copy of the main DataFrame
df_copy = df.copy()
df_copy.head()

### 9.1 Split the genres and create a new DataFrame

In [ ]:
# Split genres and create a new entry for each of the genre a movie falls into
s = df_copy['genre'].str.split(',').apply(Series, 1).stack()
# Create a temporary DataFrame with the index and genre columns
s.index = s.index.droplevel(-1)
s.name = 'genre'
# Delete the genre column
del df_copy['genre']
df_split_genres = df_copy.join(s)

In [ ]:
# Check entries (should be a lot more rows since the most movies have more than one genre)
df_split_genres.shape

In [ ]:
# This code allows pandas to show all the rows
pd.set_option('display.max_rows', None)
# Group data by genre and get mean for each genre and each variable, divide by 1 million for better visibility
df_per_genre = df_split_genres.groupby(['genre']).mean()
df_per_genre ['profit_million'] = df_per_genre['profit']/1000000
df_per_genre ['return_per_dollar'] = df_per_genre['profit']/df_per_genre['budget']
# Delete the 'profit' column
del df_per_genre['profit']
df_per_genre['worlwide_gross_income'] = df_per_genre['worlwide_gross_income']/1000000
# Delete the 'worlwide_gross_income' column
del df_per_genre['worlwide_gross_income']
df_per_genre['budget_million'] = df_per_genre['budget']/1000000
# Delete the 'budget' column
del df_per_genre['budget']
# Delete the 'year' column
del df_per_genre['year']

### 9.2 Most profitable individual genre

In [ ]:
df_per_genre.sort_values(by=['profit_million'],ascending=False).head(10)

In [ ]:
dict(df_per_genre.nlargest(5,'profit_million'))['profit_million']

In [ ]:
genre_roi = df_per_genre.nlargest(10,'profit_million')
plt.figure(figsize=(15, 5))
plt.savefig('destination_path.eps', format='eps')
genre_roi = sns.barplot(x='genre', y='profit_million', data=genre_roi.reset_index(),palette="mako")
genre_roi.set_title('Most Profitable Genres', fontdict={'fontsize':14});
genre_roi.set(xlabel='Genre', ylabel='Profit Per Dollar Invested')

### 9.3 Return per dollar invested

In [ ]:
# Sorting the value by the 'return_per_dollar'
df_per_genre.sort_values(by=['return_per_dollar'],ascending=False).head(10)

In [ ]:
dict(df_per_genre.nlargest(5,'return_per_dollar'))['return_per_dollar']

In [ ]:
genre_roi = df_per_genre.nlargest(5,'return_per_dollar')
plt.figure(figsize=(12, 5))
plt.savefig('destination_path.eps', format='eps')
genre_roi = sns.barplot(x='genre', y='return_per_dollar', data=genre_roi.reset_index(),palette="mako")
genre_roi.set_title('Most Profitable Genres Per Dollar Invested ', fontdict={'fontsize':14});
genre_roi.set(xlabel='Genre', ylabel='Profit Per Dollar Invested')

In [ ]:
# Sorting the value by the 'return_per_dollar' 
df_per_genre.sort_values(by=['return_per_dollar'],ascending=True).head(5)

In [ ]:
dict(df_per_genre.nsmallest(5,'return_per_dollar'))['return_per_dollar']

In [ ]:
genre_roi = df_per_genre.nsmallest(10,'return_per_dollar')
plt.figure(figsize=(15, 5))
genre_roi = sns.barplot(x='genre', y='return_per_dollar', data=genre_roi.reset_index(),palette="rocket")
genre_roi.set_title('Least Profitable Genres Per Dollar Invested', fontdict={'fontsize':14});
genre_roi.set(xlabel='Genre', ylabel='Profit Per Dollar Invested')


# 10. Final Takeaways and Recommendations

#### Out of the top 20 most profitable movies:
   - 18 of them are part of a franchise of movies
   - 13 of them are continuations of previous movies
   - 5 of them are the first movie of a franchise
   - 2 of them haven't had a continuation yet, but they are underway
   - Only Beauty and the Best and Titanic aren't part of a franchise yet.

    
    
#### Correlation between the data:
   - Higher budgets and higher box office are positively correlated. However, it is not guaranteed that a higher budget will bring higher revenue. It will depend on the quality and reception of the final product
   - Higher budget and higher profits have a positive moderate correlation
   - Budget and box office have a strong positive correlation
   - Duration and average vote have a moderate correlation
   - Budget and duration have a moderate correlation



#### The genres with more movies launched between 1990 and 2020 are:
   - Drama, Comedy, Horror, Thriller, Action, Biography, Adventure, Romance, and Crime.


#### The groups of genres that are doing the highest in box office revenue are:
    - Family, Fantasy, Musical       662.438628 million dollars 
    - Animation, Adventure, Drama    289.253644 million dollars
    - Action, Drama, Music           231.604044 million dollars
    - Biography, Drama, Musical      210.651842 million dollars
    - Action, Adventure, Sci-Fi      208.141854 million dollars
    
#### The group of genres that bring the highest ROI (per dollar invested) are:    
    
    - Adventure,Drama,Horror     64.225864 dollars
    - Action,Comedy,Musical      13.148727 dollars
    - Adventure,Drama,Western     9.981272 dollars
    - Action,Drama,Music          9.264162 dollars
    - Crime,Horror,Thriller       4.868740 dollars

#### The individual genres that bring the highest in box office reveue are:
    - Animation    121.788071 million dollars
    - Adventure    104.248915 million dollars
    - Sci-Fi        68.416803 million dollars
    - Action        53.555004 million dollars
    - Fantasy       49.365756 million dollars

#### The individual genre that bring the highest ROI per dolar invested are
    - Animation    1.588840 dollars
    - Musical      1.523878 dollars
    - Adventure    1.368292 dollars
    - Sci-Fi       1.188757 dollars
    - Fantasy      1.072056 dollars
    
#### The individual genre that bring the lowest ROI per dolar invested are    
    - Sport        0.323485 dollars
    - War          0.397544 dollars
    - History      0.399704 dollars
    - Crime        0.523399 dollars
    - Biography    0.591848 dollars

### Recommendations

- Invest in franchises, such as Halo and Gears.
- Invest in movies that grab all the public.
- Drama and Comedy might face higher competition since they are in larger quantities.
- Animation, Adventure, and Sci-Fi bring the highest profits. It is a good idea to invest on them.